In [2]:
import polars as pl
import matplotlib.pyplot as plt
import datetime as dt

from factor_momentum import assetspace_signal_monthly, factorspace_signals_monthly

from factor_momentum._wrappers import construct_asset_signal_monthly, _load_monthly_asset_data

/home/boobus/projects/factor_momentum/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-10-18 14:06:52,989	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
start = dt.date(2000, 1, 1)
end = dt.date(2024, 1, 1)
factor_signals = factorspace_signals_monthly(start=start, end=end, type='rolling')

factor_signals

factor,month,ret,lag_ret,signal
str,date,f64,f64,f64
"""USSLOWL_BETA""",2001-01-01,0.023647,0.02074,-0.000005
"""USSLOWL_BETA""",2001-02-01,-0.101336,0.023647,-0.001019
"""USSLOWL_BETA""",2001-03-01,-0.039318,-0.101336,-0.001153
"""USSLOWL_BETA""",2001-04-01,0.055683,-0.039318,-0.000624
"""USSLOWL_BETA""",2001-05-01,-0.014021,0.055683,-0.001393
…,…,…,…,…
"""USSLOWL_VALUE""",2023-08-01,0.010089,0.004724,0.000065
"""USSLOWL_VALUE""",2023-09-01,0.003582,0.010089,0.000153
"""USSLOWL_VALUE""",2023-10-01,-0.000703,0.003582,0.000269


In [4]:
asset_returns_and_exposures = _load_monthly_asset_data(start=start, end=end)

ColumnNotFoundError: unable to find column "in_universe"; valid columns: ["date", "rootid", "barrid", "issuerid", "instrument", "name", "cusip", "ticker", "price", "return", "specific_return", "market_cap", "price_source", "currency", "iso_country_code", "iso_currency_code", "yield", "total_risk", "specific_risk", "historical_beta", "predicted_beta", "russell_1000", "russell_2000", "daily_volume", "average_daily_volume_30", "average_daily_volume_60", "average_daily_volume_90", "bid_ask_spread", "average_daily_bid_ask_spread_30", "average_daily_bid_ask_spread_60", "average_daily_bid_ask_spread_90"]

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'sink' <---
Parquet SCAN [/home/boobus/groups/grp_quant/database/research/assets/assets_1995.parquet, ... 30 other sources]
PROJECT */31 COLUMNS

In [ ]:
benchmark = monthly_signal.group_by(['month']).agg(
    pl.col('wret').sum()
).sort('month')

benchmark

In [ ]:
binned = (monthly_signal.with_columns(
    pl.col('signal').rank('dense').over('month').alias('rank')
)
.with_columns(
    (pl.lit("p") +
    (pl.col('rank')/pl.col('rank').count().over('month')*10 - 0.0001)
    .floor()
    .cast(pl.Int32).cast(pl.Utf8))
    .alias('bin')
)
.group_by(['month', 'bin']).agg(
    pl.col('wret').sum().alias('wret')*10,
    pl.col('ret').mean().alias('ret'),
    pl.col('wret').count().alias('count')
)
.sort(['bin', 'month'])
)

ports_ew = (binned
.pivot(on='bin', index='month', values='ret')
.drop_nulls()
)

ports_vw = (binned
.pivot(on='bin', index='month', values='wret')
.drop_nulls()
)

ports_vw

In [ ]:
def plot_deciles(df, name, cum_sum=True):
    plt.figure(figsize=(8, 5))
    plt.title(name)
    dates = df['month']

    for i in range(10):
        if cum_sum:
            plt.plot(dates, df[f'p{i}'].cum_sum(),label = f'p{i}')
        else:
            plt.plot(dates, df[f'p{i}'],label = f'p{i}')
            
    plt.legend()
    plt.show()

In [ ]:
plot_deciles(ports_ew, "Cumulative Log Returns (EW)")

In [ ]:
plot_deciles(ports_vw, "Cumulative Log Returns (VW)")

In [ ]:
counts = (binned.pivot(
    on='bin', index='month', values='count'
)
)

plt.figure(figsize=(8, 5))
plt.title("Bin Counts")

for i in range(10):
    plt.plot(dates, counts[f'p{i}'], label = f'{i}')

plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
plt.title("1 Year Bin Counts")

for i in range(10):
    plt.plot(dates[:12], counts[f'p{i}'][:12], label = f'{i}')

plt.legend()
plt.show()